In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
import numpy as np
max_long = 25

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
import sys
sys.path.append('/content/gdrive/My Drive')

In [5]:
!pip install unidecode

In [5]:
import data
data.get_data, data.clean_data

(<function data.get_data>, <function data.clean_data>)

In [6]:
import token_glove
token_glove.create_list, token_glove.token_ize, token_glove.voc_token, token_glove.dict_token, token_glove.token_tweet, token_glove.glove_vector

(<function token_glove.create_list>,
 <function token_glove.token_ize>,
 <function token_glove.voc_token>,
 <function token_glove.dict_token>,
 <function token_glove.token_tweet>,
 <function token_glove.glove_vector>)

In [23]:
import metrics
metrics.recall_m, metrics.precision_m, metrics.f1_m

(<function metrics.recall_m>,
 <function metrics.precision_m>,
 <function metrics.f1_m>)

In [7]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [8]:
df = data.get_data()
cleaned_df = data.clean_data(df)

In [11]:
cleaned_df.head(1)

,id,text,target
0,1,deed reason earthquake may allah forgive u,1


In [9]:
text = token_glove.create_list(cleaned_df['text'])
token = token_glove.token_ize(text)
X = token_glove.token_tweet(text, token)

In [10]:
word_vector_matrix = token_glove.glove_vector(token)

In [12]:
cleaned_df

,id,text,target
0,1,deed reason earthquake may allah forgive u,1
1,4,forest fire near la canada,1
2,5,resident asked shelter place officer evacuatio...,1
3,6,people receive wildfire evacuation order calif...,1
4,7,got sent photo alaska smoke wildfire school,1
...,...,...,...
7608,10869,two giant crane holding bridge collapse nearby...,1
7609,10870,control wild fire california even northern par...,1
7610,10871,utckm volcano hawaii,1
7611,10872,police investigating ebike collided car little...,1


In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Activation, Dropout, LSTM
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
vocab_size = token_glove.voc_token(token)
vec_size = 200

In [33]:
y = cleaned_df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size= 0.3, stratify = y)

In [24]:
model = Sequential()

es = EarlyStopping(patience=10, restore_best_weights= True)

model.add(Embedding(vocab_size, vec_size, input_length=max_long, weights = [word_vector_matrix], trainable = True))
model.add(Conv1D(64, 8, activation ='relu'))
model.add(MaxPooling1D(2))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=Adam(learning_rate = 0.0001), loss = 'binary_crossentropy', metrics = [metrics.f1_m])
history = model.fit(X_train, y_train, batch_size=64, epochs = 400, validation_data = (X_test, y_test), callbacks = [es])

Epoch 1/400
96/96 [==============================] - 5s 46ms/step - loss: 0.6934 - f1_m: 0.4350 - val_loss: 0.6437 - val_f1_m: 0.6807
Epoch 2/400
96/96 [==============================] - 4s 42ms/step - loss: 0.6441 - f1_m: 0.5062 - val_loss: 0.5957 - val_f1_m: 0.7073
Epoch 3/400
96/96 [==============================] - 4s 46ms/step - loss: 0.6003 - f1_m: 0.5757 - val_loss: 0.5504 - val_f1_m: 0.7129
Epoch 4/400
96/96 [==============================] - 4s 42ms/step - loss: 0.5742 - f1_m: 0.6233 - val_loss: 0.5179 - val_f1_m: 0.7421
Epoch 5/400
96/96 [==============================] - 5s 47ms/step - loss: 0.5458 - f1_m: 0.6545 - val_loss: 0.4930 - val_f1_m: 0.7488
Epoch 6/400
96/96 [==============================] - 7s 76ms/step - loss: 0.5262 - f1_m: 0.6939 - val_loss: 0.4782 - val_f1_m: 0.7537
Epoch 7/400
96/96 [==============================] - 4s 42ms/step - loss: 0.5181 - f1_m: 0.6882 - val_loss: 0.4692 - val_f1_m: 0.7531
Epoch 8/400
96/96 [==============================] - 4s 42ms/s

In [25]:
model2 = Sequential()

es = EarlyStopping(patience=5,restore_best_weights= True)

model2.add(Embedding(vocab_size, vec_size, input_length=max_long, weights = [word_vector_matrix], trainable = True))
model2.add(LSTM(20))
model2.add(Dense(10, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=[metrics.f1_m])
history2 = model2.fit(X_train, y_train, batch_size=16, epochs = 400, validation_data = (X_test, y_test), callbacks = [es])

Epoch 1/400
381/381 [==============================] - 57s 117ms/step - loss: 0.5168 - f1_m: 0.6507 - val_loss: 0.4502 - val_f1_m: 0.7643
Epoch 2/400
381/381 [==============================] - 6s 16ms/step - loss: 0.4195 - f1_m: 0.7683 - val_loss: 0.4346 - val_f1_m: 0.7236
Epoch 3/400
381/381 [==============================] - 6s 16ms/step - loss: 0.3701 - f1_m: 0.7943 - val_loss: 0.4177 - val_f1_m: 0.7509
Epoch 4/400
381/381 [==============================] - 6s 15ms/step - loss: 0.3377 - f1_m: 0.8178 - val_loss: 0.4557 - val_f1_m: 0.7300
Epoch 5/400
381/381 [==============================] - 6s 15ms/step - loss: 0.3116 - f1_m: 0.8394 - val_loss: 0.4741 - val_f1_m: 0.7523
Epoch 6/400
381/381 [==============================] - 6s 16ms/step - loss: 0.2846 - f1_m: 0.8617 - val_loss: 0.4729 - val_f1_m: 0.7432
Epoch 7/400
381/381 [==============================] - 6s 15ms/step - loss: 0.2625 - f1_m: 0.8708 - val_loss: 0.4557 - val_f1_m: 0.7305
Epoch 8/400
381/381 [=========================

In [26]:
model3 = Sequential()

es = EarlyStopping(patience=5,restore_best_weights= True)

model3.add(Embedding(vocab_size, vec_size, input_length=max_long, weights = [word_vector_matrix], trainable = True))
model3.add(LSTM(20))
model3.add(Dense(10, activation='relu'))
model3.add(Dense(1, activation='sigmoid'))
model3.compile(loss='binary_crossentropy',optimizer='Adam',metrics=[metrics.f1_m])
history2 = model3.fit(X_train, y_train, batch_size=16, epochs = 400, validation_data = (X_test, y_test), callbacks = [es])

Epoch 1/400
381/381 [==============================] - 10s 22ms/step - loss: 0.5089 - f1_m: 0.6312 - val_loss: 0.4251 - val_f1_m: 0.7453
Epoch 2/400
381/381 [==============================] - 7s 20ms/step - loss: 0.3836 - f1_m: 0.7989 - val_loss: 0.4228 - val_f1_m: 0.7564
Epoch 3/400
381/381 [==============================] - 8s 20ms/step - loss: 0.3084 - f1_m: 0.8410 - val_loss: 0.4272 - val_f1_m: 0.7468
Epoch 4/400
381/381 [==============================] - 8s 20ms/step - loss: 0.2438 - f1_m: 0.8827 - val_loss: 0.5120 - val_f1_m: 0.7511
Epoch 5/400
381/381 [==============================] - 8s 20ms/step - loss: 0.1941 - f1_m: 0.9082 - val_loss: 0.6347 - val_f1_m: 0.7454
Epoch 6/400
381/381 [==============================] - 7s 19ms/step - loss: 0.1602 - f1_m: 0.9240 - val_loss: 0.6719 - val_f1_m: 0.7491
Epoch 7/400
381/381 [==============================] - 7s 19ms/step - loss: 0.1251 - f1_m: 0.9364 - val_loss: 0.7208 - val_f1_m: 0.7373


In [34]:
model4 = Sequential()

es = EarlyStopping(patience=5,restore_best_weights= True)

model4.add(Embedding(vocab_size, vec_size, input_length=max_long, weights = [word_vector_matrix], trainable = True))
model4.add(LSTM(256,return_sequences = True))
model4.add(Dropout(0.5))
model4.add(LSTM(128,return_sequences = True))
model4.add(Dropout(0.5))
model4.add(LSTM(64, return_sequences = True))
model4.add(Dropout(0.5))
model4.add(LSTM(32, return_sequences = False))
model4.add(Dropout(0.4))
model4.add(Dense(256, activation = 'relu'))
model4.add(Dropout(0.2))
model4.add(Dense(16, activation = 'relu'))
model4.add(Dropout(0.1))  
model4.add(Dense(1, activation = 'sigmoid'))
model4.compile(loss='binary_crossentropy',optimizer=Adam(learning_rate = 0.0001),metrics=[metrics.f1_m])
history2 = model3.fit(X_train, y_train, batch_size=16, epochs = 400, validation_data = (X_test, y_test), callbacks = [es])

Epoch 1/400
333/333 [==============================] - 7s 21ms/step - loss: 0.2020 - f1_m: 0.9098 - val_loss: 0.4846 - val_f1_m: 0.7858
Epoch 2/400
333/333 [==============================] - 7s 21ms/step - loss: 0.1545 - f1_m: 0.9348 - val_loss: 0.5122 - val_f1_m: 0.7886
Epoch 3/400
333/333 [==============================] - 7s 21ms/step - loss: 0.1384 - f1_m: 0.9366 - val_loss: 0.6241 - val_f1_m: 0.7761
Epoch 4/400
333/333 [==============================] - 7s 20ms/step - loss: 0.1196 - f1_m: 0.9461 - val_loss: 0.5985 - val_f1_m: 0.7663
Epoch 5/400
333/333 [==============================] - 7s 22ms/step - loss: 0.0985 - f1_m: 0.9526 - val_loss: 0.7355 - val_f1_m: 0.7745
Epoch 6/400
333/333 [==============================] - 8s 23ms/step - loss: 0.0827 - f1_m: 0.9576 - val_loss: 0.7900 - val_f1_m: 0.7665
